In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import time
import requests


import warnings
warnings.filterwarnings("ignore")

def access_token(api_secret, request_token, kite):
    user = pd.DataFrame()
    try:
        user = kite.generate_session(request_token, api_secret)
       
        print("Access token ->" + user["access_token"])
        acc_key=user["access_token"]
   
    except Exception as e:
        print("Authentication failed", str(e))
        raise

    print(user["user_name"], "has successfully signed in.")
    return user



acc_key=0

while(acc_key==0):
    user_name = "RK2267"
    password = "Shan@#2021"
#     totp_key = "RGHL2AKPKBTQUPR35WFKWDTIO5NJEALQ"
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disble-dev-shm-usage')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://kite.trade/connect/login?api_key=pv2830q1vbrhu1eu")
    time.sleep(1)
    element = driver.find_element_by_id("userid")
    element.send_keys(user_name)
    element = driver.find_element_by_id("password")
    element.send_keys(password)
    element.send_keys(Keys.RETURN)
    time.sleep(15)
#     outh = pyotp.TOTP(totp_key)
#     element = driver.find_element_by_id('externaltotp')
#     element.send_keys(outh.now())
#     element.send_keys(Keys.RETURN)
#     time.sleep(1)
    # element.close()
    print(driver.current_url)
    url = driver.current_url.split("request_token=")[1].split("&")[0]
    print(url, "new url")
    driver.quit()
    my_api_key = "pv2830q1vbrhu1eu"
    kite = KiteConnect(api_key=my_api_key)
    req_key = url
    user_details = access_token(
        "gz426yatawim9xruj2p6e51c40k44r7n", req_key, kite)
    acc_key = user_details["access_token"]
#     print("Access token ->" + acc_key)
    kite.set_access_token(acc_key)

https://kite.trade/?status=success&request_token=rDdNaqQpU2Rjd17lPv7tS7YsGcRi0lEX&action=login&type=login
rDdNaqQpU2Rjd17lPv7tS7YsGcRi0lEX new url
Access token ->Bnmmlp2PbTwgwU2CMkPGz8hj5aYb0xhW
Kakal Krishna Rao has successfully signed in.


In [3]:
from datetime import *

def history_data(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now() - timedelta(days=100))
    curr_date=datetime.now()
    if 9<=curr_date.hour<16:
        new_date=(datetime.now() - timedelta(days=1))
    else:
        new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10]+" "+'00:00:00.0')
    endd=pd.to_datetime(str(new_date)[:10]+' '+'15:30:00.0')
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print('History_data {} collected .'.format(ticker_val))
    return data1

In [11]:
from termcolor import colored
import pandas_ta as ta
def crossover(dfx,tkname):
    dfx['RSI']=ta.rsi(dfx['Close'],length=14)
    def change(s):
        return s[:10]

    dfx['Time']=dfx.Time.apply(change)
    last10=dfx.loc[len(dfx)-11:]
    buy=0
    sell=0
    nocross=0
    print('CALCULATING 10DAYS CROSSOVER OF',tkname)
    last10.reset_index(inplace=True,drop=True)
    print()
    for i in range(1,len(last10)):
        print('Date->',last10['Time'].iloc[i],'RSI->',last10['RSI'].iloc[i])
    print()
    for i in range(1,len(last10)):
        if last10['RSI'].iloc[i]>55 and last10['RSI'].iloc[i-1]<50:
            print(colored('LONG ENTRY on {}'.format(last10['Time'].iloc[i]),'green'))
            buy+=1
        elif last10['RSI'].iloc[i]<55 and last10['RSI'].iloc[i-1]>50:
            print(colored('SHORT ENTRY on {}'.format(last10['Time'].iloc[i]),'red'))
            sell+=1
        else:
            print(colored('No crossover on {}'.format(last10['Time'].iloc[i]),'blue'))
            nocross+=1

    print()
    print('--------- SUMMARY ---------------')
    print('Start date ->',last10['Time'].iloc[1])
    print('No of Long Entry :',buy)
    print('No of Short Entry :',sell)
    print('End date ->',last10['Time'].iloc[-1])
    print('----------------------------------')
    print()
    
    
    
dicct={67329: 'ATUL', 5436929: 'AUBANK', 70401: 'AUROPHARMA', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 85761: 'BALKRISIND', 87297: 'BALRAMCHIN', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 94977: 'BATAINDIA', 98049: 'BEL', 103425: 'BERGEPAINT', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2911489: 'BIOCON', 558337: 'BOSCHLTD', 134657: 'BPCL'}
tokens=[67329, 5436929, 70401, 1510401, 4267265, 4268801, 81153, 85761, 87297, 579329, 1195009, 94977, 98049, 103425, 108033, 2714625, 112129, 2911489, 558337, 134657]

for tk in tokens:
    df=history_data(tk)[0]
    crossover(df,dicct[tk])

History_data 67329 collected .
CALCULATING 10DAYS CROSSOVER OF ATUL

Date-> 2022-09-30 RSI-> 45.36444104284455
Date-> 2022-10-03 RSI-> 42.212136655644755
Date-> 2022-10-04 RSI-> 46.79139438868797
Date-> 2022-10-06 RSI-> 45.5644224371085
Date-> 2022-10-07 RSI-> 42.36272162389306
Date-> 2022-10-10 RSI-> 38.54863076962659
Date-> 2022-10-11 RSI-> 36.01498156116742
Date-> 2022-10-12 RSI-> 37.15596621010879
Date-> 2022-10-13 RSI-> 40.260230154835554
Date-> 2022-10-14 RSI-> 40.33398866589159

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No crossover on 2022-10-13
No crossover on 2022-10-14

--------- SUMMARY ---------------
Start date -> 2022-09-30
No of Long Entry : 0
No of Short Entry : 0
End date -> 2022-10-14
----------------------------------

History_data 5436929 collected .
CALCULATING 10DAYS CROSSOVER OF AUBANK

Date

History_data 579329 collected .
CALCULATING 10DAYS CROSSOVER OF BANDHANBNK

Date-> 2022-09-30 RSI-> 41.835937924853496
Date-> 2022-10-03 RSI-> 39.029486281099
Date-> 2022-10-04 RSI-> 45.39134826694249
Date-> 2022-10-06 RSI-> 48.03377080376926
Date-> 2022-10-07 RSI-> 48.59596531914046
Date-> 2022-10-10 RSI-> 44.06598889562574
Date-> 2022-10-11 RSI-> 41.93313981058249
Date-> 2022-10-12 RSI-> 44.77389230219157
Date-> 2022-10-13 RSI-> 42.90523176287483
Date-> 2022-10-14 RSI-> 43.88160317668511

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No crossover on 2022-10-13
No crossover on 2022-10-14

--------- SUMMARY ---------------
Start date -> 2022-09-30
No of Long Entry : 0
No of Short Entry : 0
End date -> 2022-10-14
----------------------------------

History_data 1195009 collected .
CALCULATING 10DAYS CROSSOVER OF BANKBAR

History_data 558337 collected .
CALCULATING 10DAYS CROSSOVER OF BOSCHLTD

Date-> 2022-09-30 RSI-> 29.41784467797697
Date-> 2022-10-03 RSI-> 26.866743576430718
Date-> 2022-10-04 RSI-> 35.64364861807539
Date-> 2022-10-06 RSI-> 36.801139366150245
Date-> 2022-10-07 RSI-> 35.44254478870108
Date-> 2022-10-10 RSI-> 33.65042407131683
Date-> 2022-10-11 RSI-> 30.758653252964873
Date-> 2022-10-12 RSI-> 33.67714176978277
Date-> 2022-10-13 RSI-> 31.768592944895634
Date-> 2022-10-14 RSI-> 30.559020306073208

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No crossover on 2022-10-13
No crossover on 2022-10-14

--------- SUMMARY ---------------
Start date -> 2022-09-30
No of Long Entry : 0
No of Short Entry : 0
End date -> 2022-10-14
----------------------------------

History_data 134657 collected .
CALCULATING 10DAYS CROSSOVER OF BPCL